In [1]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import shapely.ops
import rasterio
import rasterio.mask
import numpy
import matplotlib.pyplot
import scipy.ndimage
import scipy.interpolate
import pathlib
import pdal
import json

# Setup paths

In [2]:
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wakanae\Small_test_site')
combined_path = pathlib.Path(r'combined_data')

lidar_name = pathlib.Path(r'points.laz')
dem_name_stub = pathlib.Path(r'combined_dem')

# Read in LiDAR with relevant processing
Load in the LAZ file with relevant processing
*  Set projection: https://pdal.io/stages/filters.reprojection.html#filters-reprojection

In [3]:
crs = 2193
lidar_file_name = base_path/combined_path/lidar_name

pdal_pipeline_instructions = [
    {"type":  "readers.las", "filename": str(lidar_file_name)},
    {"type":"filters.reprojection","out_srs":"EPSG:" + str(crs)}, # reproject to NZTM
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

In [4]:
metadata=json.loads(pdal_pipeline.get_metadata())
lidar_arrays = pdal_pipeline.arrays

# Use PDAL to create and save a DEM
We can use the PDAL writers.gdal pipeline option to create a raster using GDAL drivers. This allows GDAL to be used to create a raster from scattered point cloud information - although it is not always clear which GDAL routines are being used. A primary and secondary interpolation algorithm may be emplyed depending on the 'window_size' specified. It seems the secondary interpolation approach can only be IDW.

Documentation: https://pdal.io/stages/writers.gdal.html, examples: https://pdal.io/workshop/exercises/analysis/rasterize/rasterize.html, https://pdal.io/workshop/exercises/analysis/dtm/dtm.html

Options:
* resoltuion - the resolution of the output DEM grid
* radius - the search radius during interpolation (default is sprt(2) x resolution)
* power - the power applied to the idw algorithm (https://en.wikipedia.org/wiki/Inverse_distance_weighting)
* output_type - select one or more from min, max, mean, idw, count, stdev
* window_size - the 'cell based' search distance when applying the back-up interpolation approach
* gdaldriver - specify the GDAL output file writing code. Default is GTiff. netCDF doesn't seem to be supported.
* origin_x & origin_y - define grid origin (default is None).

The following cell explores using various window, radii and IDW powers and their impact on the final raster. Each is saved out and views in QGIS. 

In [7]:
resolution = 10
dem_file_name_stub = base_path/combined_path/dem_name_stub

window_sizes = range(11,31,1)
idw_powers = range(1,3,1)
radii =  resolution * numpy.sqrt(2) * range(1,4,1)

for window_size in window_sizes:
    for idw_power in idw_powers:
        for radius in radii:

            pdal_pipeline_instructions = [
                {"type":  "writers.gdal", "resolution": resolution, "gdalopts":"a_srs=EPSG:" + str(crs),
                 "filename": str(dem_file_name_stub) + "_window_" + str(window_size) + "_power_" + str(idw_power) + "_radius_" 
                 + str(radius) + ".tiff", "output_type":["mean","idw"], "window_size": window_size, "power": idw_power, 
                 "radius": radius}
            ]

            print("Window = " + str(window_size) + ", IDW Power = " + str(idw_powers) + ", Radius = " + str(radius))

            pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), lidar_arrays)
            pdal_pipeline.execute();

[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_10_power_1_radius_14.142135623730951.tiff", "output_type": ["mean", "idw"], "window_size": 10, "power": 1, "radius": 14.142135623730951}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_10_power_1_radius_28.284271247461902.tiff", "output_type": ["mean", "idw"], "window_size": 10, "power": 1, "radius": 28.284271247461902}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_10_power_1_radius_42.42640687119285.tiff", "output_type": ["mean", "idw"], "window_size": 10, "power": 1, "radius": 42.42640687119285}]
[{"type": "writers.gdal"

[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_14_power_1_radius_42.42640687119285.tiff", "output_type": ["mean", "idw"], "window_size": 14, "power": 1, "radius": 42.42640687119285}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_14_power_2_radius_14.142135623730951.tiff", "output_type": ["mean", "idw"], "window_size": 14, "power": 2, "radius": 14.142135623730951}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_14_power_2_radius_28.284271247461902.tiff", "output_type": ["mean", "idw"], "window_size": 14, "power": 2, "radius": 28.284271247461902}]
[{"type": "writers.gdal"

[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_18_power_2_radius_42.42640687119285.tiff", "output_type": ["mean", "idw"], "window_size": 18, "power": 2, "radius": 42.42640687119285}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_19_power_1_radius_14.142135623730951.tiff", "output_type": ["mean", "idw"], "window_size": 19, "power": 1, "radius": 14.142135623730951}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_19_power_1_radius_28.284271247461902.tiff", "output_type": ["mean", "idw"], "window_size": 19, "power": 1, "radius": 28.284271247461902}]
[{"type": "writers.gdal"

[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_23_power_2_radius_14.142135623730951.tiff", "output_type": ["mean", "idw"], "window_size": 23, "power": 2, "radius": 14.142135623730951}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_23_power_2_radius_28.284271247461902.tiff", "output_type": ["mean", "idw"], "window_size": 23, "power": 2, "radius": 28.284271247461902}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_23_power_2_radius_42.42640687119285.tiff", "output_type": ["mean", "idw"], "window_size": 23, "power": 2, "radius": 42.42640687119285}]
[{"type": "writers.gdal"

[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_27_power_2_radius_42.42640687119285.tiff", "output_type": ["mean", "idw"], "window_size": 27, "power": 2, "radius": 42.42640687119285}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_28_power_1_radius_14.142135623730951.tiff", "output_type": ["mean", "idw"], "window_size": 28, "power": 1, "radius": 14.142135623730951}]
[{"type": "writers.gdal", "resolution": 10, "gdalopts": "a_srs=EPSG:2193", "filename": "C:\\Users\\pearsonra\\Documents\\data\\Wakanae\\Small_test_site\\combined_data\\combined_dem_window_28_power_1_radius_28.284271247461902.tiff", "output_type": ["mean", "idw"], "window_size": 28, "power": 1, "radius": 28.284271247461902}]
[{"type": "writers.gdal"

# Discussion
The resulting rasters show:
1. Increased IDW power produces a cleaner image when their is LiDAR, but also speckle artifact 
2. Increased radius produces increased artifact where the desnity of point data changes.
   * At the border between LiDAR and the background DEM - the LiDAR bleeds out and dominates the surrounding DEM as radius is increased.
   * At the border between the land data (LiDAR and background DEM) and the ocean data (very sparse point data), the land values propagate out without being impacted by any depth measurements (as those typically aren't very dense or close to shore).
3. Increased radius does produce increased diameter rings of values around the scattered Bathymetry sounding points.
3. Increased window values increases the distance overwhich interpolation is applied where there is no data - in the example this is around bathymetry sounding points and along the coast.

# Conclusions
* We should include zero values (or negative if at a river mouth) along the coast as otherwise the transition will drift out to sea.
* The built-in PDAL / writers.gdal is not flexible enough to cater to this interpolation use-case. 
* We will want a binning method where there is LiDAR, we will want a simple interpolaton method where there is background DEM, and we will want a smooth and highly continous approach where there is neither. 

# Other DEM options in future

### PDAL
* Write a custome PDAL python function using PDAL filters.python - https://pdal.io/stages/filters.python.html#filters-python
* Perform surface generation (i.e. filters.delaunay, filters.greedyprojection, filters.posson requires normals for each point - could do with filters.normal) then rasterise (i.e filters.faceraster) https://pdal.io/stages/filters.faceraster.html?highlight=faceraster

### GDAL
* Use GDAL grid - https://gdal.org/programs/gdal_grid.html#gdal-grid
  * Looks to be pretty limited. Supports interpolation with inverse distance to a power, inverse distance to a power with nearest neighbour searching, moving average, nearest neighbour, and linear. Linear might be appropiate for across the background DEM, and the inverse distance with powers could be used where their is LiDAR.
* There is a tutorial - https://gdal.org/tutorials/gdal_grid_tut.html

### Custom
This could of course be used as the function called in the PDAL filters.python pipeline.
* Code up a simple routine myself using a KDTree or something similar

`tree = scipy.spatial.KDTree(numpy.c_[pdal_pipeline.arrays[0]['X'].ravel(), pdal_pipeline.arrays[0]['X'].ravel()])`